# 1. Configuration and Setup

In [ ]:
import os
import requests
import geopandas as gpd
import numpy as np
import laspy
import pathlib
from shapely.geometry import Point, Polygon
from pyproj import Transformer, CRS
from ipyleaflet import Map, DrawControl
import ipywidgets as widgets

# Configuration
SHAPEFILE_ZIP = "AHN_subunits_GeoTiles.zip"
DOWNLOAD_URL = "https://geotiles.citg.tudelft.nl/AHN4_T/"
DOWNLOAD_DIR = "downloads/"
MAP_CENTER = (52.1, 5.3)
MAP_ZOOM_LEVEL = 8

# Ensure download directory exists
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# Load GeoDataFrame from a shapefile
gdf = gpd.read_file(SHAPEFILE_ZIP)


# 2. Create Interactive Map
### This section creates an interactive map using ipyleaflet and sets up coordinate transformation.

In [ ]:
# Create an interactive map centered on the specified location
m = Map(center=MAP_CENTER, zoom=MAP_ZOOM_LEVEL)
draw_control = DrawControl()

# Coordinate transformation setup
transformer = Transformer.from_crs("epsg:4326", "epsg:28992", always_xy=True)
target_crs = CRS.from_epsg(28992)  # RD New coordinate system

# Output widget for displaying messages and confirmations
output = widgets.Output()

confirm_button = widgets.Button(description="Confirm", disabled=True)
cancel_button = widgets.Button(description="Cancel", disabled=True)


# 3. Handle Draw Event
### This function handles the drawing event on the map, transforming the coordinates and finding intersecting polygons.

In [ ]:
def handle_draw(self, action, geo_json):
    global global_intersections, rd_polygon
    confirm_button.disabled = False
    cancel_button.disabled = False
    with output:
        output.clear_output()
        coordinates = geo_json['geometry']['coordinates'][0]
        rd_coordinates = [transformer.transform(lon, lat) for lon, lat in coordinates]
        rd_polygon = Polygon(rd_coordinates)
        global_intersections = gdf[gdf.intersects(rd_polygon)]

        if not global_intersections.empty:
            display(widgets.HBox([confirm_button, cancel_button]))
            print("Intersecting polygons found, confirm to download and clip data.")
        else:
            print("No intersecting polygons found.")


# 4. Confirm and Download
### This function is called when the confirm button is clicked, handling the download and clipping of the LAZ files.

In [ ]:
def on_confirm_clicked(b):
    global global_intersections, rd_polygon
    with output:
        output.clear_output()
        if global_intersections is not None:
            print("Starting the download and clipping, please wait...")
            for index, row in global_intersections.iterrows():
                url = f"{DOWNLOAD_URL}{row['GT_AHNSUB']}.LAZ"
                save_path = f"{DOWNLOAD_DIR}{row['GT_AHNSUB']}.LAZ"
                response = requests.get(url, stream=True)
                if response.status_code == 200:
                    with open(save_path, 'wb') as file:
                        file.write(response.content)

                    # Clipping the LAZ file
                    clipped_path = save_path.replace(".LAZ", "_clipped.LAZ")
                    clip_las_file(save_path, clipped_path, rd_polygon)
                    print(f"Clipped {row['GT_AHNSUB']} and saved to {clipped_path}")

            print("All operations completed.")
            confirm_button.disabled = True
            cancel_button.disabled = True
            draw_control.clear()


# 5. Clip LAZ File
### This function clips the LAZ file based on the drawn polygon.

In [ ]:
def clip_las_file(input_laz_path, output_laz_path, polygon, points_per_iter=1_000_000):
    output_dir = pathlib.Path(output_laz_path).parent
    output_dir.mkdir(parents=True, exist_ok=True)

    polygon_box = polygon.bounds

    with laspy.open(input_laz_path) as in_las:
        header = laspy.LasHeader(point_format=in_las.header.point_format, version=in_las.header.version)
        header.offsets = in_las.header.offsets  # Preserve offsets
        header.scales = in_las.header.scales  # Preserve scales
        # Create and add the CRS VLR
        wkt_vlr = laspy.vrs.known.WktCoordinateSystemVlr(target_crs.to_wkt())
        header.vlrs.append(wkt_vlr)

        with laspy.open(output_laz_path, mode='w', header=header) as out_las:
            for points in in_las.chunk_iterator(points_per_iter):
                in_bounds_mask = (
                    (points.x >= polygon_box[0]) & (points.x <= polygon_box[2]) &
                    (points.y >= polygon_box[1]) & (points.y <= polygon_box[3])
                )

                if not in_bounds_mask.any():
                    continue

                coords = np.vstack((points.x[in_bounds_mask], points.y[in_bounds_mask])).T
                inside_polygon_mask = [polygon.contains(Point(x, y)) for x, y in coords]
                filtered_points = points[in_bounds_mask][inside_polygon_mask]

                if len(filtered_points) > 0:
                    out_las.write_points(filtered_points)


# 6. Cancel Drawing
### This function handles the cancel event for drawing.

In [ ]:
def on_cancel_clicked(b):
    with output:
        output.clear_output()
        print("Polygon drawing cancelled.")
        confirm_button.disabled = True
        cancel_button.disabled = True
        draw_control.clear()


# 7. Bind Actions and Display Map
### This section binds the confirm and cancel actions to their respective buttons and displays the interactive map and output widget.

In [ ]:
# Bind actions to buttons
confirm_button.on_click(on_confirm_clicked)
cancel_button.on_click(on_cancel_clicked)

# Setup map controls
draw_control.on_draw(handle_draw)

m.add_control(draw_control)

# Display the map and output widget
display(m, output)